In [1]:
#%%
from Modele_decodeur_maison import *

from Pipeline import *
from Trainer import * 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size = 50

# Images
# images = np.load("C:/Users/lucas/Desktop/train-resnet50-res4frelu.npy")

# Texts
tokenized_fr,tokenized_en, vocab_fr,vocab_en = get_train_data_nouveau(batch_size)
n_token_fr = len(vocab_fr.keys())
n_token_en = len(vocab_en.keys())

inv_map_en = {v: k for k, v in vocab_en.items()}
inv_map_fr = {v: k for k, v in vocab_fr.items()}

n_head = 4 
num_encoder_layers = 2
num_decoder_layers = 2
dim_feedforward = 196
dropout = 0.1
activation = nn.Softmax
embedding_dim = 196

model_fr = Modèle(n_token_fr,embedding_dim,n_head, num_encoder_layers,num_decoder_layers,dim_feedforward,dropout,activation).to(device)

model_en = Modèle(n_token_en,embedding_dim,n_head, num_encoder_layers,num_decoder_layers,dim_feedforward,dropout,activation).to(device)





DEBUT_DE_PHRASE
FIN_DE_PHRASE
TOKEN_VIDE
&@@
j
;@@
DEBUT_DE_PHRASE
FIN_DE_PHRASE
TOKEN_VIDE
Ã 
&@@
j
Ã @@
Ã«
Ã«@@


In [2]:

# Y=train_auto_encoding(model_fr,tokenized_fr)
# Y=train_auto_encoding(model_en,tokenized_en)
# Y= cycle_consistency_train(model_fr,model_en,tokenized_fr,tokenized_en)
# mixed_train(model_fr,model_en,tokenized_fr,tokenized_en,200,50)



In [3]:
#%%
def tensor_to_sentence(output,inv_dic):
    result = [inv_dic[int(x)] for x in output]
    sentence = ""
    for word in result : 
        if word == "DEBUT_DE_PHRASE" :
            pass
        elif '@@' in word: 
            sentence+=word[:-2]
        elif word == "FIN_DE_PHRASE" :
            break 
        else :
            sentence+=word +" "
    return sentence

def traduit(model_A,model_B,data, inv_map):
    model_en.eval()
    model_fr.eval()
    output = torch.argmax(cycle_consistent_forward(model_A,model_B,data),dim = 2).T[j].T
    return tensor_to_sentence(output.view(-1),inv_map)

i = np.random.randint(len(tokenized_en)//50)
j = np.random.randint(50)
data,target = get_batch(tokenized_en, i)

print("Phrase à traduire : \n" + tensor_to_sentence(target.T[j],inv_map_en)+ "\n Phrase traduite : \n "+ traduit(model_en,model_fr,data, inv_map_fr))


#%%

Phrase à traduire : 
two women are sitting and talking and laughing together . 
 Phrase traduite : 
 ina brepiÃ¨ce trÃ©buaissjeundrÃ´le jeunpittoresque squsqusquimimimimimimimimimimimimimimimimimimimimatv atv atv atv imimatv atv atv imimimimimimatv atv atv imimimatv atv atv atv atv atv atv atv atv atv atv imimimkitesurkitesurkitesurkitesurkitesurkitesurkitesurkitesuratv atv atv atv atv atv atv atv atv atv atv atv atv atv atv atv atv atv atv atv atv atv 


C:\Users\valen\AppData\Local\Temp\ipykernel_1392\3639088073.py:19: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ..\aten\src\ATen\native\TensorShape.cpp:3281.)
  output = torch.argmax(cycle_consistent_forward(model_A,model_B,data),dim = 2).T[j].T


In [4]:
# import os

# import s3fs
# import pandas
# # Create filesystem object
# S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
# fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

# val_features  = np.load(fs.open('tordjx/val-resnet50-res4frelu.npy'))
# # train_features  = np.load(fs.open('tordjx/train-resnet50-res4frelu.npy'))


In [5]:
data_text, _  = get_batch(tokenized_fr, 0)
val_features = np.load("D:/Valentin/Documents/ENSAE/STATAPP/Images/val-resnet50-res4frelu.npy")
batchsz = 50
embed = model_fr.embedding(data_text)
torch.cat((embed, torch.from_numpy(val_features.reshape(1014,1024,196)[:1000].reshape(1000//batchsz ,1024,batchsz , 196)[0]).to(device))).shape

torch.Size([1121, 50, 196])

In [6]:
output = model_fr(data_text, True, torch.from_numpy(val_features.reshape(1014,1024,196)[:1000].reshape(1000//batchsz ,1024,batchsz , 196)[0]).to(device= device, dtype = torch.float32)).shape

torch.Size([97, 50, 196]) torch.Size([1024, 50, 196])
torch.Size([97, 50, 196])
torch.Size([1024, 50, 196])


RuntimeError: shape '[97, -1, 4, 49]' is invalid for input of size 10035200